## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.
---

In [ ]:
import numpy as np                 # NumPy
import cv2                         # openCV
import glob                        # Filename pattern matching
import matplotlib.pyplot as plt    # 2D plotting

# Interactive plotting in separate window
#%matplotlib qt

# Visualizations will be shown in the notebook
%matplotlib inline

## Compute the camera calibration points using chessboard images

In [ ]:
def get_3d2d_points(do_plot=False, do_file=False):
    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(8,5,0)
    objp = np.zeros((6*9, 3), np.float32)
    objp[:,:2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images
    objpoints = []                     # 3D points in real world space
    imgpoints = []                     # 2D points in image plain

    # List of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')
    print('Num of calibration images: {0}'.format(len(images)))

    # Step through the list and search for chessboard corners
    for img_id, fname in enumerate(images):
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        # http://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#findchessboardcorners
        # cv2.findChessboardCorners(image, patternSize[, corners[, flags]]) → retval, corners
        ret, corners = cv2.findChessboardCorners(gray, (9, 6), None)

        # If found - add object points, add image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            # Draw and display the corners
            cv2.drawChessboardCorners(img, (9, 6), corners, ret)
            # Draw the plot
            if do_plot:
                plt.imshow(img)
                plt.show()
            # Save to the file
            if do_file:
                write_name = 'corners_' + str(img_id) + '.jpg'
                cv2.imwrite(write_name, img)
    return objpoints, imgpoints

## Distortion correction

In [ ]:
import pickle

def pickle_dump(mtx, dist):
    # Save the camera calibration result for later use (we won't worry about rvecs / tvecs)
    dist_pickle = {}
    dist_pickle["mtx"] = mtx
    dist_pickle["dist"] = dist
    pickle.dump(dist_pickle, open('wide_dist_pickle.p', 'wb'))
    
def pickle_load():
    # Getting back the camera calibration result:
    with open('wide_dist_pickle.p', 'rb') as f:
        dist_pickle = pickle.load(f)
        return dist_pickle['mtx'], dist_pickle['dist']
    
def calibrate_camera(img):
    img_size = (img.shape[1], img.shape[0])
    
    # Do camera calibration given object points and image points
    objpoints, imgpoints = get_3d2d_points(do_plot=False, do_file=False)
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

    # Save the camera calibration result
    pickle_dump(mtx, dist)
    return mtx, dist

In [ ]:
# Test undistortion on the image
img = cv2.imread('camera_cal/calibration1.jpg')

# Calibrate camera and save data to pickle
#mtx, dist = calibrate_camera(img)

# Load calibration data from pickle
mtx, dist = pickle_load()

# Undistort image
dst = cv2.undistort(img, mtx, dist, None, mtx)
# cv2.imwrite('test_undist.jpg', dst)

# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=14)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=14)

## Color/gradient threshold

## Perspective transform

## Detect lane lines

## Determine the lane curvature